<a href="https://colab.research.google.com/github/GemmaRea/prima_prova_colab_bigquery/blob/main/importare_db_da_big_query_a_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Setup #importante per collegare notebook e bigquery
from google.colab import auth

from google.cloud import bigquery 

projectID = 'primo-progetto-8888' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
#importa librerie in colab
import pandas as pd 

import os # to save the result of queries in csv

from google.colab import syntax # highlights SQL (or other type of syntaxes) in 
                                # strings
test_string = syntax.sql('SELECT * FROM table_a WHERE condition')
test_nicer_string = syntax.sql('''SELECT * 
                                  FROM table_a 
                                  WHERE condition
                              ''')
test_nicer_string_2 = syntax.sql('SELECT * \
                                    FROM table_a \
                                    WHERE condition \
                                ')

In [3]:
%load_ext google.colab.data_table

In [6]:
#importa con id_progetto
jobID = 'bquxjob_62ff5486_18847f87ca8'  

job = client.get_job(jobID)
print(job.query)

SELECT *
 FROM `bigquery-public-data.ml_datasets.census_adult_income` LIMIT 1000


In [7]:
results = job.to_dataframe()
results.sample(5)

,age,workclass,functional_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
36,44,Private,118536,9th,5,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0,0,40,United-States,<=50K
749,55,Private,102058,10th,6,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,<=50K
446,22,Private,235853,9th,5,Never-married,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K
828,53,Self-emp-not-inc,30008,10th,6,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,35,United-States,<=50K
747,25,Private,279667,10th,6,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,45,United-States,<=50K


In [8]:
results.describe()

,age,functional_weight,education_num,capital_gain,capital_loss,hours_per_week
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,40.096,203207.533,5.486,513.984,33.724,37.852
std,16.515858,114484.921234,0.500054,5653.491508,246.659962,12.126712
min,17.0,22418.0,5.0,0.0,0.0,1.0
25%,26.0,126844.75,5.0,0.0,0.0,35.0
50%,39.0,182172.0,5.0,0.0,0.0,40.0
75%,54.0,249004.0,6.0,0.0,0.0,40.0
max,90.0,766115.0,6.0,99999.0,2231.0,99.0


In [11]:
#importa con la query di bigquery
sql_query = syntax.sql(''' SELECT *
                        FROM `bigquery-public-data.ml_datasets.census_adult_income` 
                        LIMIT 1000 ''')

df_censured = client.query(sql_query).to_dataframe()
df_censured.sample(5)

,age,workclass,functional_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
795,33,Local-gov,289716,10th,6,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
462,33,Private,87310,9th,5,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,50,United-States,<=50K
767,26,Private,252565,10th,6,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,55,United-States,<=50K
884,17,Self-emp-inc,325171,10th,6,Never-married,Other-service,Own-child,Black,Male,0,0,35,United-States,<=50K
906,18,?,152641,10th,6,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K


In [12]:
%%bigquery df --project primo-progetto-8888
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

Query is running:   0%|          |

Downloading:   0%|          |

In [13]:
#importa dataset con pandas
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=projectID, dialect='standard')

df.sample(5)

,name,count
80,Margaret,42103
54,Luis,53445
74,Nancy,43234
56,Dorothy,52647
18,Matthew,88464
